# Thai news classification based on their headlines
# AI จัดหมวดหมู่ข่าวไทยโดยใช้การพาดหัวข่าว
## by Wisarut Duangmorakot - wisarut.bank@gmail.com

- Data ได้มาจากการเขียน Script ดึงข้อมูลจากเว็บไซต์ข่าวสำนักนึงของไทย ทำให้ไม่สามารถเปิดเผยข้อมูลได้

- เราจะทำการจัดหมวดหมู่ข่าวโดยใช้การพาดหัวข่าว โดยจัดเป็น 2 หมวดหมู่คือ "ข่าวการเมือง" และ "ข่าวกีฬา" เนื่องจากข้อมูลมีขนาดใหญ่และ Computation มีจำกัด ทำให้เกิด Memory Error ขึ้นได้

- ใช้ pythainlp ในการช่วยตัดคำภาษาไทย

### Import libraries

In [1]:
import os #ใช้ในการเปิดไฟล์ทั้งหมดใน Directory ขึ้นมาอ่าน
from pythainlp import word_tokenize #ใช้ตัดคำภาษาไทย เช่น ฉันกำลังขึ้นรถไฟ -> 'ฉัน','กำลัง','ขึ้น','รถไฟ'
from sklearn.metrics import accuracy_score # ใช้เพื่อเพื่อวัดประสิทธิความแม่นยำของโมเดล
from sklearn.metrics import confusion_matrix # ใช้เพื่อเพื่อวัดประสิทธิภาพโมเดล
import re #ใช้ในการกำจัดตัวอักษรหรือคำที่ไม่ต้องการออกไป

### Import data

In [2]:
news = []
politics = 0
sports = 0

path = './politics/'
for filename in os.listdir(path): # เปิดไฟล์ทุกไฟล์ในโฟลเดอร์ politics
    f = open(path+filename,'r') # เปิดไฟล์ด้วยโหมดอ่าน
    text = f.read()
    text = text.replace(' ','') # ลบเว้นวรรคออกจากพาดหัวข่าว
    text = text.replace('"','')
    news.append(text)
    f.close()
    politics += 1
 
print (politics)    
    
path = './sports/'
for filename in os.listdir(path):
    f = open(path+filename,'r') 
    text = f.read()
    text = text.replace(' ','')
    text = text.replace('"','')
    news.append(text)
    f.close()
    sports += 1

print (sports)

60568
67790


Import Datasets เข้ามาเก็บไว้ในตัวแปร news
ซึ่ง Dataset ประกอบไปด้วยพาดหัวข่าวการเมือง 60568 samples ข่าวกีฬา 67790 samples

##### ตัวอย่างการพาดหัวข่าว

In [3]:
print (text)

ไทเกอร์หายไม่ทันหวดยูเอสโอเพ่น


### Features Extraction

In [4]:
REPLACE_NO_SPACE = re.compile("(\-)|(0)|(1)|(2)|(3)|(4)|(5)|(6)|(7)|(8)|(9)|(&quot;)|(\”)|(\")|(\“)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

news_pre = preprocess_reviews(news) # เอาสัญลักษณ์ที่ไม่เกี่ยวข้องในการ analysis ออก


train = []

for i in range(len(news_pre)):
    proc = word_tokenize(news[i], engine='newmm') #ใช้ตัดคำภาษาไทย เช่น ฉันกำลังขึ้นรถไฟ -> 'ฉัน','กำลัง','ขึ้น','รถไฟ'
    train.append(proc)
    


##### ตัวอย่าง text หลังจากผ่านการทำ Features Extraction

In [5]:
proc

['ไทเกอร์', 'หาย', 'ไม่ทัน', 'หวด', 'ยู', 'เอส', 'โอเพ่น']

### Matrix of token counts

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

def do_nothing(tokens):
    return tokens # เราไม่ใช้ tokenizer ใดๆในการตัดคำ เพราะใช้ pythainlp word_tokeniz แทนแล้ว


cv = CountVectorizer(tokenizer=do_nothing, preprocessor=None, lowercase=False)
X = cv.fit_transform(train) # apply Countvertorizer to data

y = []

for i in range(politics):
    y.append(1) #label ข่าวการเมืองด้วย 1
    
for i in range(sports):
    y.append(0) #label ข่าวกีฬาด้วย 0

เราไม่สามารถเอา text เข้า model classification ได้ทันที จะต้องเปลี่ยนจาก text แทนด้วยการนับจำนวนคำแต่ละคำ เช่น

corpus = [<br>
...     'This is the first document.',<br>
...     'This document is the second document.',<br>
...     'And this is the third one.',<br>
...     'Is this the first document?',<br>
... ]<br>

vectorizer = CountVectorizer()<br>
X = vectorizer.fit_transform(corpus)<br>
print(vectorizer.get_feature_names())<br>
['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']<br>
print(X.toarray())  <br><br>
[[0 1 1 1 0 0 1 0 1]<br>
 [0 2 0 1 0 1 1 0 1]<br>
 [1 0 0 1 1 0 1 1 1]<br>
 [0 1 1 1 0 0 1 0 1]]<br>
 
 http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.decode

In [7]:
len(train) # dataset ทั้งหมด

128358

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

ใช้ train_test_split เพื่อ random train dataset เพื่อใช้ในการเทรนโมเดล และ test dataset เพื่อใช้ในการวัดผลโมเดล

## Modeling

In [9]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 1000, n_jobs=4)
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

ใช้โมเดลมาตรฐานในการสร้างโมเดล เพื่อความรวดเร็ว ถ้านำไปใช้จริงอาจจะต้อง tune parameters ด้วย gridsearch และอาจใช้ voting method เพื่อเพิ่มประสิทธิภาพของโมเดลให้แม่นยำขึ้น

In [10]:
accuracy_score(y_test, clf.predict(X_test))

0.9761997507011531

In [11]:
confusion_matrix(y_test, clf.predict(X_test))

array([[13181,   355],
       [  256, 11880]])

โมเดลมีความแม่นยำถึง 97.62% และไม่เกิดการ overfitted ที่คลาสใดคลาสหนึ่ง

## Model testing

ทดสอบการนำโมเดลไปใช้จริง โดยลองเทสกับข่าวที่อยู่คนละสำนักพิมพ์กับที่นำข้อมูลมาเทรนและเป็นข้อมูลที่โมเดลไม่เคยเห็นมาก่อน

In [12]:
s = ['บิ๊กตู่ นิ้วมือซ้น แต่ยังฟิต ลุยถก ครม.ต่อ หลังบินกลับจากญี่ปุ่น',\
     'ครม. โยก สมชัย จากปลัดคลัง นั่งเลขาฯ สภาพัฒน์'\
     ,'กำลังใจมาเต็ม โอ๊ค ยันไม่หนีคดีฟอกเงิน โพสต์เฟซฯ เย้ยเผด็จการ'\
     ,'สื่ออังกฤษแฉ! 2 แข้งแมนฯยูไม่พอใจ โดนเปลี่ยนตัวเกมทุบนิวคาสเซิล'\
    ,'กุนซือดัตช์ตำหนิ ฟาน ไดค์ หลังเริ่มฟอร์มหลุด'\
     ,'ซีดาน เล็งคว้าแข้งจอมบุกบาร์ซา หากได้คุมแมนฯยู']

for i in range(len(s)):
    s[i] = s[i].replace(' ','')


news_pre = preprocess_reviews(s) 

news_sample = []

for i in range(len(s)):
    proc = word_tokenize(news_pre[i], engine='newmm') #ใช้ตัดคำภาษาไทย เช่น ฉันกำลังขึ้นรถไฟ -> 'ฉัน','กำลัง','ขึ้น','รถไฟ'
    print (proc)
    news_sample.append(proc)

news_sample = cv.transform(news_sample)
clf.predict(news_sample)

['บิ๊ก', 'ตู่', 'นิ้วมือ', 'ซ้น', 'แต่', 'ยัง', 'ฟิต', 'ลุย', 'ถก', 'ครม.', 'ต่อ', 'หลัง', 'บิน', 'กลับ', 'จาก', 'ญี่ปุ่น']
['ครม.', 'โยก', 'สม', 'ชัย', 'จาก', 'ปลัด', 'คลัง', 'นั่ง', 'เลขาฯ', 'สภาพัฒน์']
['กำลัง', 'ใจมา', 'เต็ม', 'โอ๊ค', 'ยัน', 'ไม่', 'หนี', 'คดี', 'ฟอกเงิน', 'โพสต์', 'เฟซฯ', 'เย้ย', 'เผด็จการ']
['สื่อ', 'อังกฤษ', 'แฉ', 'แข้ง', 'แมนฯยู', 'ไม่พอใจ', 'โดน', 'เปลี่ยนตัว', 'เกม', 'ทุบ', 'นิวคาสเซิล']
['กุนซือ', 'ดัตช์', 'ตำหนิ', 'ฟาน', 'ได', 'ค์', 'หลัง', 'เริ่ม', 'ฟอร์ม', 'หลุด']
['ซี', 'ดาน', 'เล็ง', 'คว้า', 'แข้ง', 'จอม', 'บุก', 'บาร์', 'ซา', 'หาก', 'ได้', 'คุม', 'แมนฯยู']


array([1, 1, 1, 0, 0, 0])

โมเดลสามารถทำงานได้อย่างถูกต้อง (0 แทนข่าวกีฬา, 1 แทนข่าวการเมือง)

## Conclusion

- สามารถสร้างโมเดลที่ทำนายประเภทของข่าวได้โดยใช้การพาดหัว ซึ่งมีความแม่นยำ 97.62%
- สามารถเพิ่มประสิทธิภาพของโมเดลได้ โดยการ tune model เช่นใช้ Gridsearch หรือ การ voting ของโมเดล
- การไม่นำข้อมูลที่เกิดขึ้นบ่อยเกินหรือน้อยเกินมาสร้างโมเดลก็ทำให้โมเดลดีขึ้นได้เช่นกัน

# Reference
- http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.decode
- https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184
- https://stackoverflow.com/questions/18262293/how-to-open-every-file-in-a-folder
- https://www.pythonforbeginners.com/files/reading-and-writing-files-in-python